Firstly we have to intall PostgreSql database adapter(Driver) for the Python programming language(pip install psycopg2-binary)

In [1]:
pip install --upgrade pip

Requirement already up-to-date: pip in /Users/a1/anaconda3/lib/python3.7/site-packages (20.0.2)
Note: you may need to restart the kernel to use updated packages.


In [2]:
# import packages

import psycopg2
import getpass
import numpy as np
import datetime
import time
import pandas as pd
import dateutil.relativedelta as dr

# connect with database
a = input('User of database: ' )
b = input('Name of database: ' )
c = input('Host: ' )
d = input('Port: ' )
conn=psycopg2.connect(user='postgres',
                      database='postgres',
                      host='localhost',
                      port='5432',
                      password=getpass.getpass('Insert password: '))
                        
print(conn.get_dsn_parameters())
# check parameters


User of database: postgres
Name of database: postgres
Host: localhost
Port: 5432
Insert password: ········
{'user': 'postgres', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'tty': '', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


In [3]:
# extract data from bond_description and quotes

query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM public.quotes  
                  GROUP BY "ISIN") AS a
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM public.bond_description 
                       WHERE "Status"!='Погашен') AS b
            ON a."ISIN" = b."ISINCode";"""

# read query in python

data = pd.read_sql_query(query, conn) 

# Extract more specified data 
data.drop(['ISINCode'], axis=1)
data.BUYBACKPRICE = data.BUYBACKPRICE.fillna(0)
data = data[~data.CPN.isna()].reset_index(drop = True)
data = data[data.CPN != 0]
data = data[data.CouponPerYear > 0]

# if you had timestamp format in any of extracted fields so change format to date
# in my case it was only buybackdate

data.BUYBACKDATE = data.BUYBACKDATE
data.head()

,CPN,ISIN,BUYBACKDATE,BUYBACKPRICE,ISINCode,FaceValue,HaveOffer,AmortisedMty,CouponPerYear,NumCoupons,EndMtyDate,CouponType,IssuerName
0,42.3836,RU000A0JUPY1,2017-06-23,$100.00,RU000A0JUPY1,1000.0,True,False,2,20,2024-06-14,Переменный,Банк Зенит
1,43.1315,RU000A0JUEE7,2019-01-22,$100.00,RU000A0JUEE7,1000.0,True,False,2,20,2024-01-16,Переменный,ВЭБ-лизинг
2,66.0700,RU000A0JTY40,2016-06-01,$100.00,RU000A0JTY40,1000.0,True,False,2,10,2018-05-30,Переменный,Банк Зенит
3,59.3400,RU000A0JV227,2017-12-07,$100.00,RU000A0JV227,1000.0,True,False,2,20,2024-11-28,Переменный,Роснефть НК
4,59.8356,RU000A0JT8N3,2016-10-21,$100.00,RU000A0JT8N3,1000.0,True,False,2,14,2019-10-18,Переменный,Почта России


In [4]:
# function to count coupon payments

def coupon_count(coup_str):
    full_matrix = pd.DataFrame() # local var
    end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
    a = int(coup_str.CouponPerYear.values[0])
    b = float(coup_str.NumCoupons.values[0])
    types = True
    if coup_str.CPN is not None or coup_str.CPN != 0:
        if coup_str.AmortisedMty.astype(str).values[0] == 'False':
            types = False
            amount = coup_str.CPN.values[0]
            for j in np.linspace((b - 1) / a, 0, b):
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25 * j))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount += coup_str.FaceValue.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                amount = coup_str.CPN.values[0]
        else:
            percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
            index = (b - 1)
            face = coup_str.FaceValue.values[0]
            while index >= 0:
                amount = percent * face
                date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                    date = end_date
                    amount = coup_str.BUYBACKPRICE.values[0]
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                    date = end_date
                    amount = face
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    break
                full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                index -= 1
                face = face * (1 - percent)
    my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                           , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
    full_matrix = pd.DataFrame(full_matrix.values, index = my_index, columns = full_matrix.columns)
    return full_matrix

In [5]:
# full_matrix contains coupon payments for all the emitents
# global var:

full_matrix = pd.DataFrame()

# fill in full_matrix

for i in data.iterrows():
    full_matrix = pd.concat([full_matrix, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
    
# example
full_matrix.fillna(0).head(20)

/Users/a1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.
  del sys.path[0]


,,,2006-08-19,2006-10-06,2007-01-05,2007-02-18,2007-04-06,2007-06-11,2007-06-15,2007-06-16,2007-07-07,2007-08-11,...,2031-03-21,2031-09-20,2032-03-21,2032-09-19,2033-03-21,2033-09-20,2034-03-21,2034-09-20,2035-03-21,2035-09-20
ISIN,Amortisation,Issuer,,,,,,,,,,,,,,,,,,,,,
RU000A0JUPY1,Not Amortised,Банк Зенит,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JUEE7,Not Amortised,ВЭБ-лизинг,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTY40,Not Amortised,Банк Зенит,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JV227,Not Amortised,Роснефть НК,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JT8N3,Not Amortised,Почта России,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JPTJ4,Not Amortised,МТС,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JU7X1,Not Amortised,Банк Зенит,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JU823,Amortised,Воронежская Обл Прав,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
RU000A0JTZB0,Not Amortised,ЮТэйр-Финанс,0.0,0.0000,0.0000,0.0,0.0000,0.0,0.0,0.0,0.0000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [6]:
# check
print(full_matrix.values[0].T != 'nan')
print(full_matrix.shape) # table: 500 rows, 2230 columns



[ True  True  True ...  True  True  True]
(500, 2230)


In [7]:
# from the given date to the end
import datetime
from datetime import datetime, timedelta, date
def from_date_matrix(issuer_name, date):
    
    spec_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name), \
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) < date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            spec_payments = pd.concat([spec_payments, data], axis = 1)
    return spec_payments

In [8]:
# example
# insert date in format yyyy-mm-dd
from_date_matrix('Банк ВТБ','2010-01-01')

,,2010-01-05,2010-04-06,2010-07-06,2010-10-06,2011-01-05,2011-04-06,2011-07-07,2011-10-06,2012-01-05,2012-04-06,...,2016-01-05,2016-01-06,2016-01-19,2016-02-16,2016-03-24,2016-03-30,2016-06-23,2016-06-28,2016-06-29,2016-07-08
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JNGU3,Not Amortised,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,...,30.2918,$100.00,0.00,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JU773,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,35.28,0.00,0,35.28,$100.00
RU000A0JTK20,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,1020.32,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JTN01,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,1019.7,0.00,0.00,0.00,0,0.00,0
RU000A0JUQE1,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,23.81,0.00,23.81,$100.00,0.00,0
RU000A0JV3Q3,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,0.00,0.00,0,0.00,0


In [9]:
# payments on specified date
def on_spec_date(issuer_name, date):
    on_spec_date_payments = pd.DataFrame()
    for i in range(0,len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name),\
        level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365 / 12 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how='all').fillna(0).columns[i].split('-')[2])
        if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365 / 12 + int(date.split('-')[2]) == date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            on_spec_date_payments = pd.concat([on_spec_date_payments, data], axis = 1)
    return on_spec_date_payments

In [10]:
on_spec_date('Банк ВТБ','2016-02-16')

,,2016-02-16
ISIN,Amortisation,
RU000A0JNGU3,Not Amortised,0.0
RU000A0JU773,Not Amortised,0.0
RU000A0JTK20,Not Amortised,0.0
RU000A0JTN01,Not Amortised,1019.7
RU000A0JUQE1,Not Amortised,0.0
RU000A0JV3Q3,Not Amortised,0.0


In [11]:
# from the beginning date to today

import datetime
acctual_date = datetime.datetime.now().strftime("%Y-%m-%d")

def from_date_to_today(issuer_name):
    from_date_to_today_payments = pd.DataFrame()
    for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
        date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
        how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + \
        int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365/12 \
        + int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
        if int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[0]) * 365 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[1]) * 365/12 + \
        int(datetime.datetime.now().strftime("%Y-%m-%d").split('-')[2]) > date_size:
            data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
            from_date_to_today_payments = pd.concat([from_date_to_today_payments, data], axis = 1)
    return from_date_to_today_payments

In [12]:
from_date_to_today('Банк ВТБ')

,,2006-10-06,2007-01-05,2007-04-06,2007-07-07,2007-10-06,2008-01-05,2008-04-06,2008-07-06,2008-10-05,2009-01-05,...,2016-01-05,2016-01-06,2016-01-19,2016-02-16,2016-03-24,2016-03-30,2016-06-23,2016-06-28,2016-06-29,2016-07-08
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JNGU3,Not Amortised,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,...,30.2918,$100.00,0.00,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JU773,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,35.28,0.00,0,35.28,$100.00
RU000A0JTK20,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,1020.32,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JTN01,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,1019.7,0.00,0.00,0.00,0,0.00,0
RU000A0JUQE1,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,23.81,0.00,23.81,$100.00,0.00,0
RU000A0JV3Q3,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,0.00,0.00,0,0.00,0


In [13]:
# BONUS: from the given date to the end

def deli(issuer_name, date, db_login, from_date_matrix):

    conn = psycopg2.connect(user = db_login['user'],
                            database = db_login['database'], 
                            host = db_login['host'], 
                            port = db_login['port'], 
                            password = getpass.getpass('Insert password: '))

    query = """SELECT *
            FROM (SELECT MAX("CPN") AS "CPN", "ISIN", max("BUYBACKDATE") AS "BUYBACKDATE",
                  max("BUYBACKPRICE") AS "BUYBACKPRICE"
                  FROM public.quotes  
                  GROUP BY "ISIN") AS a
            INNER JOIN(SELECT "ISINCode", "FaceValue", "HaveOffer", "AmortisedMty", 
                       "CouponPerYear", "NumCoupons", "EndMtyDate", "CouponType", "IssuerName"
                       FROM public.bond_description 
                       WHERE "Status"!='Погашен') AS b
            ON a."ISIN" = b."ISINCode";"""

    # read query in python

    data = pd.read_sql_query(query, conn) 

    # Extract more specified data 
    data.drop(['ISINCode'], axis=1)
    data.BUYBACKPRICE = data.BUYBACKPRICE.fillna(0)
    data = data[~data.CPN.isna()].reset_index(drop = True)
    data = data[data.CPN != 0]
    data = data[data.CouponPerYear > 0]

    # if you had timestamp format in any of extracted fields so change format to date
    # in my case it was only buybackdate

    data.BUYBACKDATE = data.BUYBACKDATE
    
    def coupon_count(coup_str):
        full_matrix = pd.DataFrame() # local var
        end_date = coup_str.BUYBACKDATE.values[0] if coup_str.BUYBACKDATE.values[0] is not None else coup_str.EndMtyDate.values[0]
        a = int(coup_str.CouponPerYear.values[0])
        b = float(coup_str.NumCoupons.values[0])
        types = True
        if coup_str.CPN is not None or coup_str.CPN != 0:
            if coup_str.AmortisedMty.astype(str).values[0] == 'False':
                types = False
                amount = coup_str.CPN.values[0]
                for j in np.linspace((b - 1) / a, 0, b):
                    date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25 * j))
                    if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                        date = end_date
                        amount = coup_str.BUYBACKPRICE.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                        date = end_date
                        amount += coup_str.FaceValue.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    amount = coup_str.CPN.values[0]
            else:
                percent = float(coup_str.CPN.values[0]) / float(coup_str.FaceValue.values[0])
                index = (b - 1)
                face = coup_str.FaceValue.values[0]
                while index >= 0:
                    amount = percent * face
                    date = coup_str.EndMtyDate.values[0] - dr.relativedelta(days = round(365.25*index/a))
                    if date >= end_date and coup_str.EndMtyDate.values[0] != end_date :
                        date = end_date
                        amount = coup_str.BUYBACKPRICE.values[0]
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    elif date >= end_date and coup_str.EndMtyDate.values[0] == end_date:
                        date = end_date
                        amount = face
                        full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                        break
                    full_matrix = pd.concat([full_matrix, pd.DataFrame({date.strftime("%Y-%m-%d"): [amount]})], axis = 1)
                    index -= 1
                    face = face * (1 - percent)
        my_index = pd.MultiIndex.from_tuples([(coup_str.ISIN.values[0], 'Amortised' if types else 'Not Amortised'
                                               , coup_str.IssuerName.values[0] )], names=['ISIN', 'Amortisation', 'Issuer'])
        full_matrix = pd.DataFrame(full_matrix.values, index = my_index, columns = full_matrix.columns)
        return full_matrix
    
    full_matrix = pd.DataFrame()

    # fill in full_matrix

    for i in data.iterrows():
        full_matrix = pd.concat([full_matrix, coupon_count(pd.DataFrame(i[1]).T)], sort = True)
        full_matrix = full_matrix.fillna(0)
    
    import datetime
    def from_date_matrix(issuer_name, date):
        spec_payments = pd.DataFrame()
        for i in range(0, len(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns)):
            date_size = int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, \
            how = 'all').fillna(0).columns[i].split('-')[0]) * 365 + int(full_matrix.xs(str(issuer_name), \
            level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[1]) * 365/12 + \
            int(full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).columns[i].split('-')[2])
            if int(date.split('-')[0]) * 365 + int(date.split('-')[1]) * 365/12 + int(date.split('-')[2]) < date_size:
                data = full_matrix.xs(str(issuer_name), level = 2).dropna(axis = 1, how = 'all').fillna(0).iloc[:, i:i+1]
                spec_payments = pd.concat([spec_payments, data], axis = 1)
        return spec_payments
    
    results = {'ISSUER': issuer_name, 'DATE': date, 'MATRIX_OF_PAYMENTS': from_date_matrix(issuer_name, date)}
    return results

In [14]:
a = input('User of database: ' )
b = input('Name of database: ' )
c = input('Host: ' )
d = input('Port: ' )
login = {'user': a, 'database': b, 'host': c, 'port': d}
x = deli('Банк ВТБ','2010-01-01', login, from_date_matrix)
print(x['ISSUER'], x['DATE'])
x['MATRIX_OF_PAYMENTS'].loc[:, (x['MATRIX_OF_PAYMENTS'] != 0).any(axis = 0)]

User of database: postgres
Name of database: postgres
Host: localhost
Port: 5432
Insert password: ········


/Users/a1/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:48: DeprecationWarning: object of type <class 'float'> cannot be safely interpreted as an integer.


Банк ВТБ 2010-01-01


,,2010-01-05,2010-04-06,2010-07-06,2010-10-06,2011-01-05,2011-04-06,2011-07-07,2011-10-06,2012-01-05,2012-04-06,...,2016-01-05,2016-01-06,2016-01-19,2016-02-16,2016-03-24,2016-03-30,2016-06-23,2016-06-28,2016-06-29,2016-07-08
ISIN,Amortisation,,,,,,,,,,,,,,,,,,,,,
RU000A0JNGU3,Not Amortised,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,30.2918,...,30.2918,$100.00,0.00,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JU773,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,35.28,0.00,0,35.28,$100.00
RU000A0JTK20,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,1020.32,0.0,0.00,0.00,0.00,0,0.00,0
RU000A0JTN01,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,1019.7,0.00,0.00,0.00,0,0.00,0
RU000A0JUQE1,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,23.81,0.00,23.81,$100.00,0.00,0
RU000A0JV3Q3,Not Amortised,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,...,0.0000,0,0.00,0.0,0.00,0.00,0.00,0,0.00,0


# For some reason table in Github seems a little strange form, while in Jupyter notebook everything okey!